# Map Visualizations for CoW & UCDP/PRIO

by Jenna Jordan

In [1]:
import pandas as pd
import geopandas as gpd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import geoplot
import ipywidgets as widgets
from ipywidgets import interact

In [2]:
cow_war_shapes = gpd.read_file("./cow_wars_shapes.gpkg")
ucdp_war_shapes = gpd.read_file("./ucdp_wars_shapes.gpkg")
cow_war_data = pd.read_csv("./cow_wars_data.csv")
ucdp_war_data = pd.read_csv("./ucdp_wars_data.csv")

In [3]:
cow_war_gdf = cow_war_shapes.merge(cow_war_data, on='FEATUREID')
ucdp_war_gdf = ucdp_war_shapes.merge(ucdp_war_data, on='FEATUREID')

In [4]:
ucdp_war_gdf

,FEATUREID,geometry,month,gw_id,iso_alpha3,cow_id,country,border_change,war_count,month_count,year,war_count_cum
0,117,"POLYGON ((24.31971 43.69664, 24.30778 43.69749...",1946-01-01,360,ROU,360,Romania,1.0,0,0,1946,0
1,117,"POLYGON ((24.31971 43.69664, 24.30778 43.69749...",1946-02-01,360,ROU,360,Romania,0.0,0,1,1946,0
2,117,"POLYGON ((24.31971 43.69664, 24.30778 43.69749...",1946-03-01,360,ROU,360,Romania,0.0,0,2,1946,0
3,117,"POLYGON ((24.31971 43.69664, 24.30778 43.69749...",1946-04-01,360,ROU,360,Romania,0.0,0,3,1946,0
4,117,"POLYGON ((24.31971 43.69664, 24.30778 43.69749...",1946-05-01,360,ROU,360,Romania,0.0,0,4,1946,0
...,...,...,...,...,...,...,...,...,...,...,...,...
128867,245,"POLYGON ((34.09223 9.47747, 33.90162 9.48121, ...",2016-02-01,625,SDN,625,Sudan,0.0,1,841,2016,492
128868,245,"POLYGON ((34.09223 9.47747, 33.90162 9.48121, ...",2016-03-01,625,SDN,625,Sudan,0.0,1,842,2016,493
128869,245,"POLYGON ((34.09223 9.47747, 33.90162 9.48121, ...",2016-04-01,625,SDN,625,Sudan,0.0,1,843,2016,494
128870,245,"POLYGON ((34.09223 9.47747, 33.90162 9.48121, ...",2016-05-01,625,SDN,625,Sudan,0.0,1,844,2016,495


In [5]:
cow_month_by_month = cow_war_gdf.groupby("month_count")
ucdp_month_by_month = ucdp_war_gdf.groupby("month_count")

## Wars through time & space: Correlates of War vs UCDP/PRIO

The Correlates of War (CoW) and UCDP/PRIO are the two most dominant data sources for conflict and peace researchers. This visualization illustrates some of these differences - both use different country identifier systems and have slightly different methodologies for recording wars. The maps show how many wars have occured in a given country during a given month, according to either CoW or UCDP/PRIO.

In [6]:
play = widgets.Play(
    interval=1000,
    value=0,
    min=0,
    max=844,
    step=1,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider(min=0, max=844, description='month', value=0, layout={'width': '1000px'})
widgets.jslink((play, 'value'), (slider, 'value'))
play

Play(value=0, description='Press play', interval=1000, max=844)

## Ongoing Wars Per Month

In [7]:
@interact(month_count=slider)
def plot_mnth(month_count):

    norm = mpl.colors.Normalize(vmin=0,vmax=4)
    fig, (ax1, ax2) = plt.subplots(2)
    plt.subplots_adjust(hspace=.05)
    this_month = pd.Timestamp(year=1946, month=1, day=1) + pd.DateOffset(months=month_count)
    this_month_text = this_month.strftime('%Y %b')
    
    plt.suptitle(this_month_text, size=30)
    ax1.set_title('Correlates of War')
    ax2.set_title('UCDP/PRIO')
    
    cow_df = cow_month_by_month.get_group(month_count)
    cow_map = geoplot.choropleth(cow_df, ax=ax1,
                            hue='war_count', 
                            cmap='magma', 
                            norm=norm, 
                            figsize=(20,10), 
                            legend=True,
                            edgecolor='white', 
                            linewidth=0.75)

    ucdp_df = ucdp_month_by_month.get_group(month_count)
    ucdp_map = geoplot.choropleth(ucdp_df, ax=ax2,
                            hue='war_count', 
                            cmap='magma', 
                            norm=norm, 
                            figsize=(20,10), 
                            legend=True,
                            edgecolor='white', 
                            linewidth=0.75)
    return None

interactive(children=(IntSlider(value=0, description='month', layout=Layout(width='1000px'), max=844), Output(…

## Cumulative number of months spent at war (since 1946)

In [8]:
@interact(month_count=slider)
def plot_mnth(month_count):

    norm = mpl.colors.Normalize(vmin=0,vmax=1600)
    fig, (ax1, ax2) = plt.subplots(2)
    plt.subplots_adjust(hspace=.05)
    this_month = pd.Timestamp(year=1946, month=1, day=1) + pd.DateOffset(months=month_count)
    this_month_text = this_month.strftime('%Y %b')
    
    plt.suptitle(this_month_text, size=30)
    ax1.set_title('Correlates of War')
    ax2.set_title('UCDP/PRIO')
    cow_df = cow_month_by_month.get_group(month_count)
    cow_map = geoplot.choropleth(cow_df, ax=ax1,
                            hue='war_count_cum', 
                            cmap='plasma', 
                            norm=norm, 
                            figsize=(20,10), 
                            legend=True,
                            edgecolor='white', 
                            linewidth=0.75)

    ucdp_df = ucdp_month_by_month.get_group(month_count)
    ucdp_map = geoplot.choropleth(ucdp_df, ax=ax2,
                            hue='war_count_cum', 
                            cmap='plasma', 
                            norm=norm, 
                            figsize=(20,10), 
                            legend=True,
                            edgecolor='white', 
                            linewidth=0.75)
    return None

interactive(children=(IntSlider(value=0, description='month', layout=Layout(width='1000px'), max=844), Output(…

## Polity Score: Autocracies and Democracies

Polity IV gives each country-year in the CoW system a "polity" score, which is the "democracy" score minus the "autocracy" score. -10 is the most autocratic score, +10 is the most democratic score, and 0 means either a neutral score or that the government is in a transition period.

In [9]:
@interact(month_count=slider)
def plot_mnth(month_count):
    fig, ax = plt.subplots()
    norm = mpl.colors.Normalize(vmin=-10,vmax=10)
    plt.rcParams["figure.figsize"] = (20,15)
    this_month = pd.Timestamp(year=1946, month=1, day=1) + pd.DateOffset(months=month_count)
    this_month_text = this_month.strftime('%Y %b')
    
    plt.suptitle(this_month_text, size=30)
    
    cow_df = cow_month_by_month.get_group(month_count)
    cow_map = geoplot.choropleth(cow_df, ax=ax,
                            hue='polity2', 
                            cmap='RdYlBu', 
                            norm=norm, 
                            figsize=(20,10), 
                            legend=True, 
                            legend_kwargs={'orientation': 'horizontal'},
                            edgecolor='white', 
                            linewidth=0.75)

    return None

interactive(children=(IntSlider(value=0, description='month', layout=Layout(width='1000px'), max=844), Output(…

## Citations

### Datasets used

#### CShapes

Weidmann, Nils B., Doreen Kuse, and Kristian Skrede Gleditsch. 2010. The Geography of the International System: The CShapes Dataset. International Interactions 36 (1).

#### Correlates of War

Correlates of War Project. 2017. "State System Membership List, v2016." Online, http://correlatesofwar.org
Sarkees, Meredith Reid and Frank Wayman (2010). Resort to War: 1816 - 2007. Washington DC: CQ Press.

Palmer, Glenn, Vito D'Orazio, Michael R. Kenwick, and Roseanne W. McManus. Forthcoming. “Updating the Militarized Interstate Dispute Data: A Response to Gibler, Miller, and Little.” International Studies Quarterly.

#### UCDP/PRIO Armed Conflict

Pettersson, Therese; Stina Högbladh & Magnus Öberg, 2019. Organized violence, 1989-2018 and peace agreements, Journal of Peace Research 56(4).

Gleditsch, Nils Petter, Peter Wallensteen, Mikael Eriksson, Margareta Sollenberg, and Håvard Strand (2002) Armed Conflict 1946-2001: A New Dataset. Journal of Peace Research 39(5).

#### Polity IV

Monty G. Marshall, Ted Robert Gurr, Keith Jaggers (2018). POLITY IV PROJECT: Dataset Users' Manual. Center for Systemic Peace.

### Data Transformation

For the original wrangling and normalization of the CoW datasets, see: https://github.com/jenna-jordan/international-relations-database-extended/blob/master/Wrangle_Data/CoW_Normalize.ipynb

For the original wrangling and normalization of the UCDP/PRIO dataset, see: https://github.com/jenna-jordan/international-relations-database-extended/blob/master/Wrangle_Data/UCDP-PRIO_Normalize.ipynb

For the geospatial wrangling and creation of the GeoPackages loaded in this notebook, see: https://github.com/jenna-jordan/IS590DV-FinalProject/blob/master/Notebooks_Part3/Create%20Data%20for%20War%20Maps.ipynb